In [ ]:
import numpy as np
import cv2
from cv2 import aruco
import glob
import time

from src.calibration.new.utils import get_chessboard_coordinates
        
# extra imports
import os
import matplotlib.pyplot as plt

In [ ]:
# helper funciton to quickly show an image
def imshow(img, scale=1):
    (h, w) = img.shape[0:2]
    aspect = w/h
    default_w = 6.4 #inches
    w2 = scale*default_w
    h2 = scale*default_w/aspect
    figsize = (w2, h2)
    f = plt.figure(figsize=figsize)
    
    if len(img.shape) > 2:
        img_bgr = img
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        plt.imshow(img_rgb)
    elif np.min(img) == 0 and np.max(img) == 1:
        #binary image
        img_binary = img
        plt.imshow(img_binary, cmap='gray', vmin=0, vmax=1)
    else: 
        #grayscale image
        img_gray = img
        plt.imshow(img_gray, cmap='gray',  vmin=0, vmax=255)

def imshow_cv2(img, timeout=5000):
    cv2.imshow('img', img)
    cv2.waitKey(timeout)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [ ]:
# constants for this run
# provided as function/module arguments
chessboard_rows = 6
chessboard_cols = 9
chessboard_square_mm = 23

# this may be inferred by # of camera folders
n_cameras = 6

# these may be inferred by dims of 1st image
width = 1920
height = 1200
n_channels = 3

# optionally show every image in opencv to check detected marker positions (much slower)
show_img = False

project_folder = "/Users/caxon/olveczky/dannce_data/test_chris_hannah/intrinsics"

In [ ]:
# intermediate values

num_vertices = chessboard_cols * chessboard_rows
n_cameras = 6

chessboard_points = get_chessboard_coordinates(
    chessboard_rows=chessboard_rows,
    chessboard_cols=chessboard_cols,
    square_size_mm=chessboard_square_mm,
)

object_vertex_coords = np.tile(chessboard_points, (n_cameras, 1, 1))

# Run calibration for all cameras (slow)

In [ ]:
# load raw images into memory
raw_images_all = []
n_images_all = []

for camera_idx in range(n_cameras):
    camera_name = f"Camera{camera_idx+1}"
    print(f"Processing {camera_name}")
    
    glob_pattern = f"{project_folder}/{camera_name}/*.tiff"

    # list of filepaths for calibration images
    img_filepaths = sorted(glob.glob(glob_pattern))
    n_images = len(img_filepaths)
    n_images_all.append(n_images)

    # load all images and store them in a numpy array

    # this array should be about 500MB to 1GB
    raw_images = np.zeros((n_images, height, width, n_channels), dtype=np.uint8)
    raw_images_all.append(raw_images)

    print(f"Loading {n_images} into memory. May take a few seconds")
    for idx, img_filepath in enumerate(img_filepaths):
        this_img = cv2.imread(img_filepath)
        raw_images[idx] = this_img
    
    print(f"Done loading {camera_name}\n")

print("-- Done loading all images")

In [ ]:
camera_params = []
objpoints_all = []
imgpoints_all = []

for camera_idx in range(n_cameras):
    camera_name = f"Camera{camera_idx+1}"
    print(f"Calibrating {camera_name}")

    raw_images = raw_images_all[camera_idx]
    n_images = n_images_all[camera_idx]
    
    objpoints = [] # 3d point in real world space
    objpoints_all.append(objpoints)
    imgpoints = [] # 2d points in image plane.
    imgpoints_all.append(imgpoints)

    start = time.perf_counter()
    
    for img_idx in range(n_images):
        this_img = raw_images[img_idx,:,:,:].copy()
        gray = cv2.cvtColor(this_img, cv2.COLOR_BGR2GRAY)
        
        # Find the chess board corners
        # 2nd param is Size: (Width, Height)
        success, corner_coords = cv2.findChessboardCorners(gray, (chessboard_cols, chessboard_rows), None)
        
        if success == True:
            imgpoints.append(corner_coords)
            objpoints.append(chessboard_points)
            
            if show_img is True:
                cv2.drawChessboardCorners(this_img, (chessboard_cols, chessboard_rows), corner_coords, True)
                imshow_cv2(this_img, timeout=5_000)
        else:
            print("Failure: Image #", img_idx)
    
    end = time.perf_counter()
    
    print(f"Found all corners in {(end-start)*1000:.2f} ms [{n_images} images]")
    
    rpe, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1] , None, None, flags=cv2.CALIB_FIX_K3)
    
    #unpack distortion coefficients (ignore k3)
    k1, k2, p1, p2 = dist[0,0:4]
    
    camera_params.append({
        'n': camera_idx,
        'camera_matrix': mtx,
        'rdist': [k1, k2],
        'tdist': [p1, p2],
        'dist': dist,
        'rpe' : rpe
    })
    
    print(f"Successfully calibrated {camera_name}\n")

print("Done!")

In [ ]:
imshow_cv2(raw_images[0])

# Unproject image to test the camera matrix

In [ ]:
camera_idx = 5
image_idx = 20

this_img = raw_images_all[camera_idx][image_idx,:,:,:]
h, w = this_img.shape[:2]
param = camera_params[camera_idx]
mtx= param['camera_matrix']
dist = param['dist']

imgpoints = imgpoints_all[camera_idx][image_idx]
objpoints = objpoints_all[camera_idx][image_idx]

# new_camera_mtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

# 5th arg optional: new_camera_mtx
undistorted_image = cv2.undistort(this_img, mtx, dist, None, None)
imshow(undistorted_image, scale=3)

In [ ]:
u = undistorted_img.copy()
cv2.line(u, imgpoints[0,0,:].astype(np.uint), imgpoints[8,0,:].astype(np.uint), (255,255,255), 2, cv2.LINE_AA)
None;


In [ ]:
d = this_img.copy()
pts = imgpoints[:,0,:].astype(np.uint)
for pt in pts:
    undist_pt = cv2.undistortPoints
    # print(f"Drawing: {pt[0]} {pt[1]} ")
    cv2.circle(d, (pt[0],pt[1]), radius=3, color=(0, 0, 255), thickness=-1)
print("Done")
# imshow(d, scale =3)

In [ ]:
d = this_img.copy()
pts = imgpoints[:,0,:].astype(np.uint)
for i in range(len(objpoints)):
    # print(f"Drawing: {pt[0]} {pt[1]} ")
    this_pt_distorted = imgpoints[i,0,:]
    this_pt_undistorted = cv2.undistortPoints(this_pt_distorted, mtx, dist).squeeze()
    this_pt_undistorted[0] = (this_pt_undistorted[0])*w
    this_pt_undistorted[1] = (this_pt_undistorted[1])*h
    print(this_pt_undistorted)
    cv2.circle(d, (this_pt_undistorted).astype(np.uint64), radius=3, color=(0, 0, 255), thickness=-1)
print("Done")
imshow(d, scale =2)

In [ ]:
this_pt_distorted

In [ ]:
pts = np.array([[901.7, 141.6]], dtype=np.float32)
cv2.undistortPoints(pts, mtx, dist)


In [ ]:
(objpoints[0]).shape


In [ ]:
imshow(raw_images[0])

In [ ]:
# sample intrinsics matrix for Camera #1

camera_matrix=np.array([[2.32446619e+03, 0.00000000e+00, 9.69888498e+02],
 [0.00000000e+00, 2.32791287e+03, 5.95493637e+02],
 [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
r_distort=np.array([-0.07290682, 0.09718552])
t_distort = np.array([-0.00195815, -0.00363516])

dist = np.hstack((r_distort, t_distort, 0))

In [ ]:
this_img = undistorted_img.copy()
gray = cv2.cvtColor(undistorted_img.copy(), cv2.COLOR_BGR2GRAY)

success, corner_coords = cv2.findChessboardCorners(gray, (9, 6), None)
print(success)

corner_img = this_img.copy()
cv2.drawChessboardCorners(corner_img, (9, 6), corner_coords, True)
print("CORNER IMG")
imshow(corner_img, 2)
err, mtx ,dist ,rvecs, tvecs = cv2.calibrateCamera(
    [objpoints], [corner_coords], (1920, 1200), None, None, flags=cv2.CALIB_FIX_K3
)

In [ ]:
imshow(raw_images[0]

In [ ]:
for i in range(4):

    print( "\n\nIMAGE #", i)
    this_img = raw_images_all[0][i]
    objpoints = get_chessboard_coordinates(chessboard_rows=6, chessboard_cols=9, square_size_mm=23)
    
    gray = cv2.cvtColor(this_img.copy(), cv2.COLOR_BGR2GRAY)
    
    success, corner_coords = cv2.findChessboardCorners(gray, (9, 6), None)
    
    corner_img = this_img.copy()
    cv2.drawChessboardCorners(corner_img, (9, 6), corner_coords, True)
    
    imshow(corner_img, 1)
    err, mtx ,dist ,rvecs, tvecs = cv2.calibrateCamera(
        [objpoints], [corner_coords], (1920, 1200), None, None, flags=cv2.CALIB_FIX_K3
    )
    
    print("CAMERA MATRIX:\n", mtx)
    print("DIST:\n", str(dist))
    print("T_VECS:\n", str(rvecs[0]))
    print("R_VECS:\n", str(tvecs[0]))
    print("ERR:\n", str(err))


In [ ]:
np.set_printoptions(suppress=True, precision=4)

In [ ]:
print(str(rvecs[0]))